**Summary**: This notebook functions as a development and testing ground for the `ogma.py` data coordinator. It orchestrates the management of all necessary data inputs and outputs for the suite of applications within `CapitalLogic`.

*Version*: 0.1

---

<p align="center">
<b style="font-size:50px">CapitalLogic</b><br>
<b style="font-size:36px">Ogma &ndash; Data Orchastrator</b>
</p>

<img src="../../../images/Ogma.jpeg" style="float: center;"/>

---

<p style="font-size:25px" align="center"><b>To do's</b></p>

- [ ] Do test each input
- [ ] Implement a cache system

---

# 📄 Description

Ogma serves as the pivotal system for orchestrating all critical data flows within the `CapitalLogic` suite, mirroring the functionality of a traditional database manager. This strategic setup paves the way for an eventual migration to a more advanced lakehouse data architecture.

## Data Inputs

Ogma's role in managing inputs is visualized through a mind map, which clearly delineates the data's organization and flow paths:

<!--
Mermaid
  - Documentation: https://mermaid.js.org/
  - Live Editor (for producing images): https://mermaid.live/

This is the Mermaid code:

mindmap
  root["`**Ogma**`"]
    ::icon(fa fa-keyboard)
    Regulatory
    ::icon(fa fa-folder)
      The Grid
      ::icon(fa fa-file-excel)
      ["BoE/PRA
      *Risk-Free Curves*"]
      ::icon(fa fa-file-excel)
      ["BoE/PRA
      *PD/FS/COD*"]
      ::icon(fa fa-file-excel)
    Liabilities
    ::icon(fa fa-folder)
      Liabilities
      ::icon(fa fa-file-excel)
    Securities
    ::icon(fa fa-folder)
      Securities
      ::icon(fa fa-file-excel)
    Configuration File
    ::icon(fa fa-file-text)
-->

[![](https://mermaid.ink/img/pako:eNqdksFqwzAMQH_F6LSFlt5z25Jll0FLs9s8qBorqWhsF8UeDaX_voSthW2Flt1s6elJAh2g8oYgBcvOWNxpp5R4H940rJJk3lhMkpWG9zGuVJpy5d1djarG6Zb6tUcx91-5JTWxxeClv8DWvjUk36RSrxtSz8Lm9P_JcktT2lfUnvlhmkf_NFssH06RZMnddloIkcqifFCXnIf8l26Rz4pyls3z2z0vjGtuOTB11zf-A1-zl1RFuVH-m73mzryruYmCgb1TxZC-1GOsCrQPQxFMwJJYZDNcymGENYQNWdKQDk-DstWg3XHgMAZf9q6CNEikCcSdwUA5YyNoIa2x7ej4CSeLwr4?type=png)](https://mermaid.live/edit#pako:eNqdksFqwzAMQH_F6LSFlt5z25Jll0FLs9s8qBorqWhsF8UeDaX_voSthW2Flt1s6elJAh2g8oYgBcvOWNxpp5R4H940rJJk3lhMkpWG9zGuVJpy5d1djarG6Zb6tUcx91-5JTWxxeClv8DWvjUk36RSrxtSz8Lm9P_JcktT2lfUnvlhmkf_NFssH06RZMnddloIkcqifFCXnIf8l26Rz4pyls3z2z0vjGtuOTB11zf-A1-zl1RFuVH-m73mzryruYmCgb1TxZC-1GOsCrQPQxFMwJJYZDNcymGENYQNWdKQDk-DstWg3XHgMAZf9q6CNEikCcSdwUA5YyNoIa2x7ej4CSeLwr4)

## Application-Specific Data Handling

Ogma customizes data collection based on the specific needs of each application within the suite. It organizes data by application, maintaining folders that archive the history of executed cases. This feature allows users to effortlessly track and compare outcomes across different scenarios.

## Data Outputs

Applications within the suite are tasked with depositing their outputs in designated folders as dictated by Ogma. This structured approach enables Ogma to consolidate both inputs and outputs, offering a comprehensive dataset for analysis. Users can then employ tools such as Excel, PowerBI, or Tableau for in-depth examination and insights.

## Programming Approach

Our programming methodology emphasizes efficiency, particularly in handling dataframes with Pandas. Traditional loop operations, which tend to be less efficient, are minimized in favor of vectorized operations. This choice significantly boosts processing speed and performance, though it may occasionally impact code readability. This balance between efficiency and clarity is a key consideration in our development process.

## References

- [Mermaid](https://mermaid.js.org/) &ndash; diagramming and charting tool
- [xbbg](https://xbbg.readthedocs.io/en/latest/) &ndash; intuitive Bloomberg data API

# 🛫 Preamble

In [ ]:
# Required libraries
import base64
import datetime
import os
import shutil
import sys
import tempfile
import warnings
from collections import namedtuple
from typing import Tuple

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
import yaml
from dateutil.relativedelta import relativedelta
from IPython.core.display import HTML
from IPython.display import display
from openpyxl import load_workbook

sys.path.append(f"{os.getcwd()}/../../src/ogma")

from modules.applications import CLLBalanceSheetOptimization
from modules.cl.investment_standards import get_single_name_limits
from modules.cl.liabilities import create_yearly_liabilities_cash_flows
from modules.cl.risk_free_curves import (
    create_monthly_risk_free_curves,
    create_yearly_risk_free_curves,
)
from modules.cl.securities import (
    create_yearly_securities_cash_flows,
    set_securities_adj_cash_flows,
    set_securities_licat_c1,
    set_securities_scr,
)
from modules.db.liabilities import check_cll_liabilities, get_cll_liabilities
from modules.db.regulatory import (
    get_cll_licat_c1_c3,
    get_cll_pod_fs_cod,
    get_cll_risk_free_curves,
    get_cll_solvency_capital_ratios,
)
from modules.db.securities import check_cll_securities, get_cll_securities
from modules.exceptions import OgmaError

pd.set_option("display.float_format", "{:.3f}".format)
pd.set_option("display.max_columns", None)

# 🔧 Case Configuration File

In [ ]:
SAVE_INPUTS_FILES = True  #🅾️
# Load the case configuration file
CUR_CFG_FILE = [
        "../../Data/data v0.2/CapitalLogic/CasesCfg/ExcelUIDev.yml"  # 0
][0]  #🅾️
with open(CUR_CFG_FILE, "r") as ymlfile:
    case_cfg = yaml.load(ymlfile, Loader=yaml.SafeLoader)

In [ ]:
# Identify the application
if case_cfg["Application"] == "CLLBalanceSheetOptimization":
    application = CLLBalanceSheetOptimization(
        case_cfg["Case"],
        case_cfg["Description"],
        case_cfg["ValuationDate"],
        case_cfg["DBRootFolder"],
        case_cfg["Schema"],
    )
    print(
        f"🌐 Running application {type(application).__name__} on case "
        + f"'{application.case}'"
    )
else:
    raise OgmaError(f"Doesn't recognize application {case_cfg['Application']}")

# 💾 Reading the Inputs

In [ ]:
print("💾 Reading the inputs ...")

verbose = True
ignore_cache = False
test = True

# Since the dataframes are interdependent, at this moment we update all of them
# whenever any of the input files has changed.
if ignore_cache or (
    # Risk-free curves
    (
        not os.path.isfile(application.get_risk_free_curves_fn("yearly") + ".lz4")
        or os.path.getmtime(application.get_regulatory_risk_free_curves_fn())
        > os.path.getmtime(application.get_risk_free_curves_fn("yearly") + ".lz4")
    )
    # SCR
    or (
        not os.path.isfile(application.get_securities_attr_fn() + ".lz4")
        or os.path.getmtime(application.get_regulatory_the_grid_fn())
        > os.path.getmtime(application.get_securities_attr_fn() + ".lz4")
    )
    # POD, FS, and COD
    or (
        not os.path.isfile(application.get_securities_cash_flows_fn("yearly") + ".lz4")
        or os.path.getmtime(application.get_regulatory_pod_fs_cod_fn())
        > os.path.getmtime(application.get_securities_cash_flows_fn("yearly") + ".lz4")
    )
    # LICAT
    or (
        not os.path.isfile(application.get_securities_attr_fn() + ".lz4")
        or (
            os.path.getmtime(application.get_regulatory_licat_fn())
            > os.path.getmtime(application.get_securities_attr_fn() + ".lz4")
        )
    )
    or (
        not os.path.isfile(application.get_licat_c3_fn() + ".lz4")
        or (
            os.path.getmtime(application.get_regulatory_licat_fn())
            > os.path.getmtime(application.get_licat_c3_fn() + ".lz4")
        )
    )
    # Liabilities
    or (
        not os.path.isfile(application.get_liabilities_cash_flows_fn("yearly") + ".lz4")
        or os.path.getmtime(application.get_liabilities_fn())
        > os.path.getmtime(application.get_liabilities_cash_flows_fn("yearly") + ".lz4")
    )
    # Securities
    or (
        not os.path.isfile(application.get_securities_attr_fn() + ".lz4")
        or os.path.getmtime(application.get_securities_fn())
        > os.path.getmtime(application.get_securities_attr_fn() + ".lz4")
    )
):
    cached_regulatory_risk_free_curves = False
    cached_solvency_capital_ratios = False
    cached_pod_fs_cod = False
    cached_licat = False
    cached_liabilities = False
    cached_securities = False
else:
    cached_regulatory_risk_free_curves = True
    cached_solvency_capital_ratios = True
    cached_pod_fs_cod = True
    cached_licat = True
    cached_liabilities = True
    cached_securities = True


if not cached_regulatory_risk_free_curves:
    if verbose:
        print("   ▸ Risk-Free Curves")
    df_inputs_cll_risk_free_curves = get_cll_risk_free_curves(
        application.get_regulatory_risk_free_curves_fn()
    )

if not cached_solvency_capital_ratios:
    if verbose:
        print("   ▸ Solvency Capital Ratios")
    df_inputs_cll_scr = get_cll_solvency_capital_ratios(
        application.get_regulatory_the_grid_fn()
    )

if not cached_pod_fs_cod:
    if verbose:
        print("   ▸ SII POD, FS, and COD")
    df_inputs_cll_pod_fs_cod = get_cll_pod_fs_cod(
        application.get_regulatory_pod_fs_cod_fn()
    )

if not cached_licat:
    if verbose:
        print("   ▸ LICAT")
    df_inputs_cll_licat_c1, df_inputs_cll_licat_c3 = get_cll_licat_c1_c3(
        application.get_regulatory_licat_fn()
    )

if not cached_liabilities:
    if verbose:
        print("   ▸ Liabilities")
    (
        df_inputs_cll_liabilities_funds,
        df_inputs_cll_liabilities_linked_funds,
        df_inputs_cll_liabilities_cash_flows,
    ) = get_cll_liabilities(application.get_liabilities_fn())
    test = check_cll_liabilities(
        df_inputs_cll_liabilities_funds, df_inputs_cll_liabilities_cash_flows, True,
    )

if not cached_securities:
    if verbose:
        print("   ▸ Securities")
    (
        df_inputs_cll_securities_attr,
        df_inputs_cll_securities_funds,
        df_inputs_cll_securities_sec2funds,
        df_inputs_cll_securities_cash_flows,
    ) = get_cll_securities(application.get_securities_fn())
    test = test and check_cll_securities(
        df_inputs_cll_securities_attr, df_inputs_cll_securities_sec2funds, True
    )

# 🧩 Processing the inputs for CapitalLogic

In [ ]:
print("🧩 Processing the inputs for CapitalLogic ...")

print("   ▸ Monthly & Yearly Risk-Free Curves")
if not cached_regulatory_risk_free_curves:
    application.df_monthly_risk_free_curves = pd.merge(
        create_monthly_risk_free_curves(
            df_inputs_cll_risk_free_curves, application.valuation_date
        ),
        df_inputs_cll_liabilities_cash_flows.groupby(["Period"])
        .count()
        .reset_index()[["Period"]],
        left_on=["Month"],
        right_on=["Period"],
        how="inner",
    )[["Month", "SpotRate", "ForwardRate", "Duration", "DiscountFactor"]]
    application.df_yearly_risk_free_curves = create_yearly_risk_free_curves(
        df_inputs_cll_risk_free_curves, application.valuation_date
    )
else:
    application.df_monthly_risk_free_curves = feather.read_feather(
        application.get_risk_free_curves_fn("monthly") + ".lz4"
    )
    application.df_yearly_risk_free_curves = feather.read_feather(
        application.get_risk_free_curves_fn("yearly") + ".lz4"
    )

print("   ▸ Liabilities")
if not cached_liabilities:
    application.df_liabilities_funds = df_inputs_cll_liabilities_funds
    application.df_monthly_liabilities_cash_flows = (
        df_inputs_cll_liabilities_cash_flows.copy()
    )
    application.df_monthly_liabilities_cash_flows.rename(
        columns={"Period": "Month"}, inplace=True
    )
    application.df_monthly_liabilities_cash_flows = pd.merge(
        application.df_monthly_liabilities_cash_flows,
        application.df_monthly_risk_free_curves[["Month"]],
        on=["Month"],
        how="inner",
    )
    application.df_yearly_liabilities_cash_flows = create_yearly_liabilities_cash_flows(
        df_inputs_cll_liabilities_cash_flows,
        df_inputs_cll_risk_free_curves,
        application.valuation_date,
    )
    # Align the horizon of the yearly risk-free curve with the liabilities one
    application.df_yearly_risk_free_curves = pd.merge(
        application.df_yearly_risk_free_curves,
        application.df_yearly_liabilities_cash_flows.groupby(["Year"])
        .count()
        .reset_index()[["Year"]],
        on=["Year"],
        how="inner",
    )
else:
    application.df_liabilities_funds = feather.read_feather(
        application.get_liabilities_funds_fn() + ".lz4",
    )
    application.df_monthly_liabilities_cash_flows = feather.read_feather(
        application.get_liabilities_cash_flows_fn("monthly") + ".lz4",
    )
    application.df_yearly_liabilities_cash_flows = feather.read_feather(
        application.get_liabilities_cash_flows_fn("yearly") + ".lz4",
    )


print("   ▸ Securities")
if not cached_securities:
    application.df_securities_attr = set_securities_scr(
        df_inputs_cll_securities_attr, df_inputs_cll_scr
    )
    application.df_monthly_securities_cash_flows = set_securities_adj_cash_flows(
        df_inputs_cll_securities_attr,
        df_inputs_cll_securities_cash_flows,
        df_inputs_cll_pod_fs_cod,
        df_inputs_cll_risk_free_curves,
        application.valuation_date,
    )
    application.df_yearly_securities_cash_flows = pd.merge(
        create_yearly_securities_cash_flows(
            application.df_monthly_securities_cash_flows,
            df_inputs_cll_risk_free_curves,
            application.valuation_date,
        ),
        application.df_yearly_risk_free_curves[["Year"]],
        on=["Year"],
        how="inner",
    )
    application.df_securities_attr = set_securities_licat_c1(
        application.df_securities_attr,
        application.df_yearly_securities_cash_flows,
        application.df_yearly_risk_free_curves,
        df_inputs_cll_licat_c1,
    )
    if len(application.df_yearly_securities_cash_flows["Year"].unique()) != len(
        application.df_yearly_risk_free_curves
    ):
        raise OgmaError(
            "The cash flows from securities do not fully align with the entire "
            + "horizon of the liabilities."
        )
    # We take the opportunity to set LICAT C3 here
    application.df_licat_c3 = df_inputs_cll_licat_c3
    if len(application.df_licat_c3) != len(application.df_yearly_risk_free_curves):
        raise OgmaError(
            "The LICAT C3 table does not fully align with the entire horizon of "
            + "the liabilities."
        )
else:
    application.df_securities_attr = feather.read_feather(
        application.get_securities_attr_fn() + ".lz4",
    )
    application.df_monthly_securities_cash_flows = feather.read_feather(
        application.get_securities_cash_flows_fn("monthly") + ".lz4",
    )
    application.df_yearly_securities_cash_flows = feather.read_feather(
        application.get_securities_cash_flows_fn("yearly") + ".lz4",
    )
    application.df_licat_c3 = feather.read_feather(
        application.get_licat_c3_fn() + ".lz4",
    )

print("   ▸ Investment standards")
if not cached_securities:
    application.df_single_name_limits = get_single_name_limits(
        application.df_securities_attr
    )
else:
    application.df_single_name_limits = feather.read_feather(
        application.get_single_name_limits_fn() + ".lz4",
    )

# 📁 Savings the processed inputs

In [ ]:
print(
    "📁 Saving the processed inputs to the folder related to case "
    + f"'{application.case}' ..."
)
# Create the folder that stores the processed inputs file if it doesn’t exist.
if not os.path.exists(application.get_processed_inputs_folder_name()):
    os.mkdir(application.get_processed_inputs_folder_name())

if not cached_regulatory_risk_free_curves:
    # Risk-free curves
    feather.write_feather(
        application.df_monthly_risk_free_curves,
        application.get_risk_free_curves_fn("monthly") + ".lz4",
    )
    feather.write_feather(
        application.df_yearly_risk_free_curves,
        application.get_risk_free_curves_fn("yearly") + ".lz4",
    )

if not cached_licat:
    # LICAT C3
    feather.write_feather(
        application.df_licat_c3, application.get_licat_c3_fn() + ".lz4",
    )

if not cached_liabilities:
    # Liabilities
    feather.write_feather(
        application.df_liabilities_funds,
        application.get_liabilities_funds_fn() + ".lz4",
    )
    feather.write_feather(
        df_inputs_cll_liabilities_linked_funds,
        application.get_liabilities_linked_funds_fn() + ".lz4",
    )
    feather.write_feather(
        application.df_monthly_liabilities_cash_flows,
        application.get_liabilities_cash_flows_fn("monthly") + ".lz4",
    )
    feather.write_feather(
        application.df_yearly_liabilities_cash_flows,
        application.get_liabilities_cash_flows_fn("yearly") + ".lz4",
    )

if not cached_securities:
    # Securities
    feather.write_feather(
        application.df_securities_attr, application.get_securities_attr_fn() + ".lz4",
    )
    feather.write_feather(
        df_inputs_cll_securities_funds, application.get_securities_funds_fn() + ".lz4",
    )
    feather.write_feather(
        df_inputs_cll_securities_sec2funds,
        application.get_securities_sec2funds_fn() + ".lz4",
    )
    feather.write_feather(
        application.df_monthly_securities_cash_flows,
        application.get_securities_cash_flows_fn("monthly") + ".lz4",
    )
    feather.write_feather(
        application.df_yearly_securities_cash_flows,
        application.get_securities_cash_flows_fn("yearly") + ".lz4",
    )
    # Investment standards
    feather.write_feather(
        application.df_single_name_limits,
        application.get_single_name_limits_fn() + ".lz4",
    )

# Parameters
with open(application.get_parameters_fn() + ".yml", "w",) as file:
    yaml.safe_dump(case_cfg["Parameters"], file)

# ⚙️ Running the BSO model

In [ ]:
# Create the YAML file for the Julia model
tmp_yml_fn = f"{tempfile.gettempdir()}/_cll_bs_opt_inputs.yml"
with open(tmp_yml_fn, "w") as file:
    yaml.dump(
        {
            "case": application.case,
            "description": application.description,
            "inputs_path": f"{application.get_processed_inputs_folder_name()}/",
            "indent": 2,
           "simple_fixed_point": False,
            "timer": True,
            "verbose": True,
        },
        file,
    )

# Run the optimization model
print("⚙️  Calling Julia ...")
if (
    os.system(
        "julia "
        + f'"../../src/cll/bso/bso.jl" '
        + f'"{tmp_yml_fn}"'
    )
    != 0
):
    print("❗️ The model didn't execute properly.")
    sys.exit(1)

# 📊 Creating Excel File

In [ ]:
# Results from the model
df_results_securities = feather.read_feather(
    f"{application.get_processed_inputs_folder_name()}/results_securities.lz4"
)
df_results_securities_portfolios = feather.read_feather(
    f"{application.get_processed_inputs_folder_name()}/results_securities_portfolios.lz4"
)
df_results_liabilities_portfolios = feather.read_feather(
    f"{application.get_processed_inputs_folder_name()}/results_liabilities_portfolios.lz4"
)
df_results_cash_flows_annual_periods = feather.read_feather(
    f"{application.get_processed_inputs_folder_name()}/results_cash_flows_annual_periods.lz4"
)
df_results_cash_flows_monthly_periods = feather.read_feather(
    f"{application.get_processed_inputs_folder_name()}/results_cash_flows_monthly_periods.lz4"
)
df_results_scenarios = feather.read_feather(
    f"{application.get_processed_inputs_folder_name()}/results_scenarios.lz4"
)

In [ ]:
if True:
    if case_cfg["Parameters"]["Execution"]["AuditMode"]:
        TEMPLATE_FILENAME = "Capital Logic Results Template - Extended.xlsx"
    else:
        TEMPLATE_FILENAME = "Capital Logic Results Template - Summary.xlsx"

    # Create the Excel file
    RESULTS_TEMPLATE_FILENAME = (
        f"{application.get_processed_inputs_folder_name()}/../{TEMPLATE_FILENAME}"
    )
    RESULTS_FILENAME = (
        f"{application.get_processed_inputs_folder_name()}/"
        + f"{df_results_scenarios['Scenario'][0]}"
        + f" - {datetime.datetime.now().strftime('%d-%m-%Y-%Hh%M')}"
        + ".xlsx"
    )
    shutil.copyfile(
        RESULTS_TEMPLATE_FILENAME, RESULTS_FILENAME,
    )
    with pd.ExcelWriter(
        f"{application.get_processed_inputs_folder_name()}/"
        + f"{df_results_scenarios['Scenario'][0]}"
        + f" - {datetime.datetime.now().strftime('%d-%m-%Y-%Hh%M')}"
        + ".xlsx",
        mode="a",
        if_sheet_exists="overlay",
    ) as writer:
        # Scenarios
        df_results_scenarios.to_excel(writer, sheet_name="_Scenarios", index=False)
        # Securities
        df = pd.merge(
            application.df_securities_attr,
            df_results_securities,
            on=["HoldingID"],
            how="inner",
        )
        df.insert(0, "Scenario", df.pop("Scenario"))
        df.to_excel(writer, sheet_name="_Securities", index=False)
        # Securities by portfolio
        if case_cfg["Parameters"]["Execution"]["AuditMode"]:
            df = pd.merge(
                application.df_securities_attr,
                df_results_securities_portfolios,
                on=["HoldingID"],
                how="inner",
            )
            df.insert(0, "Portfolio", df.pop("Portfolio"))
            df.insert(0, "Scenario", df.pop("Scenario"))
            df["ReportingClass1"] = df["ReportingClass1"].fillna("Other")
            df["ReportingClass1_Portfolio"] = df.apply(
                lambda x: x.ReportingClass1 + "-" + x.Portfolio, axis=1
            )
            df["CALMAssetType_Portfolio"] = df.apply(
                lambda x: x.CALMAssetType + "-" + x.Portfolio, axis=1
            )
            df["CurHoldingMV"] = df["CurHolding"] * df["Price"] / 100
            df["OptHoldingMV"] = df["OptHolding"] * df["Price"] / 100
            df["CurHoldingSCRMV"] = df[
                ["Portfolio", "CurHoldingMV", "MASCR", "NonMASCR"]
            ].apply(
                lambda x: (
                    x["CurHoldingMV"] * x["MASCR"]
                    if x["Portfolio"] == "A"
                    else x["CurHoldingMV"] * x["NonMASCR"]
                ),
                axis=1,
            )
            df["OptHoldingSCRMV"] = df[
                ["Portfolio", "OptHoldingMV", "MASCR", "NonMASCR"]
            ].apply(
                lambda x: (
                    x["OptHoldingMV"] * x["MASCR"]
                    if x["Portfolio"] == "A"
                    else x["OptHoldingMV"] * x["NonMASCR"]
                ),
                axis=1,
            )
            df.to_excel(writer, sheet_name="_SecuritiesPortfolios", index=False)
        # Liabilities
        df = pd.merge(
            df_results_liabilities_portfolios,
            application.df_liabilities_funds,
            on=["FundID"],
            how="inner",
        )[
            [
                "Scenario",
                "FundID",
                "Portfolio",
                "CurAlloc",
                "OptAlloc",
                "CurMAShare",
                "CurVAShare",
                "CurRFShare",
                "MaxMAShare",
                "MaxVAShare",
                "CurCF",
                "OptCF",
                "InterestUp",
                "InterestDown",
                "InflationUp",
                "InflationDown",
            ]
        ]
        df["FundID_Portfolio"] = df.apply(
            lambda x: x.FundID + "-" + x.Portfolio, axis=1
        )
        df.to_excel(writer, sheet_name="_Liabilities", index=False)
        # Cash flows by period
        if case_cfg["Parameters"]["Execution"]["AuditMode"]:
            pd.merge(
                df_results_cash_flows_annual_periods,
                application.df_licat_c3,
                on=["Year"],
                how="inner",
            ).to_excel(writer, sheet_name="_CashFlowsAnnualPeriods", index=False)
            df_results_cash_flows_monthly_periods.to_excel(
                writer, sheet_name="_CashFlowsMonthlyPeriods", index=False
            )

# 🛝 Playground

## Cash Flows to Excel

In [ ]:
if False:
    with pd.ExcelWriter(
        "/Users/robinduquette/Tmp/securities_cash_flows.xlsx"
    ) as writer:
        application.df_monthly_securities_cash_flows.pivot(
            index="Month", columns="HoldingID", values="CashFlow"
        ).fillna(0).reset_index().to_excel(
            writer, sheet_name="CashFlow", index=False,
        )
        application.df_monthly_securities_cash_flows.pivot(
            index="Month", columns="HoldingID", values="PODAdjCashFlow"
        ).fillna(0).reset_index().to_excel(
            writer, sheet_name="PODAdjCashFlow", index=False,
        )
        application.df_monthly_securities_cash_flows.pivot(
            index="Month", columns="HoldingID", values="FSAdjCashFlow"
        ).fillna(0).reset_index().to_excel(
            writer, sheet_name="FSAdjCashFlow", index=False,
        )